In [1]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import util

from args import get_test_args
from collections import OrderedDict
from json import dumps
from models import BiDAF
from os.path import join
from tensorboardX import SummaryWriter
from tqdm import tqdm
from ujson import load as json_load
from util import collate_fn, SQuAD



In [2]:

model_path = 'save/train/baseline-01/step_50048.pth.tar'
word2idx_path = 'data/word2idx.json'

# python test.py --split SPLIT --load_path PATH --name NAME
# python test.py --split dev --load_path save/train/baseline-01/step_50048.pth.tar --name first

In [8]:
# arguments

import argparse
from args import add_common_args, add_train_test_args

parser = argparse.ArgumentParser('Test a trained model on SQuAD')

add_common_args(parser)
add_train_test_args(parser)

parser.add_argument('--split',
                    type=str,
                    default='dev',
                    choices=('train', 'dev', 'test'),
                    help='Split to use for testing.')
parser.add_argument('--sub_file',
                    type=str,
                    default='submission.csv',
                    help='Name for submission file.')
parser.add_argument('--para_limit',
                    type=int,
                    default=400,
                    help='Max number of words in a paragraph')
parser.add_argument('--ques_limit',
                    type=int,
                    default=50,
                    help='Max number of words to keep from a question')
parser.add_argument('--ans_limit',
                    type=int,
                    default=30,
                    help='Max number of words in a training example answer')
# Require load_path for test.py
params = '--load_path save/train/baseline-01/best.pth.tar --name eval'.split()
args = parser.parse_args(params)

# Set up logging
args.save_dir = util.get_save_dir(args.save_dir, args.name, training=False)
log = util.get_logger(args.save_dir, args.name)
log.info(f'Args: {dumps(vars(args), indent=4, sort_keys=True)}')
device, gpu_ids = util.get_available_devices()
args.batch_size *= max(1, len(gpu_ids))


[04.02.21 00:27:43] Args: {
    "ans_limit": 30,
    "batch_size": 64,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "hidden_size": 100,
    "load_path": "save/train/baseline-01/best.pth.tar",
    "max_ans_len": 15,
    "name": "eval",
    "num_visuals": 10,
    "num_workers": 4,
    "para_limit": 400,
    "ques_limit": 50,
    "save_dir": "./save/test/eval-01",
    "split": "dev",
    "sub_file": "submission.csv",
    "test_eval_file": "./data/test_eval.json",
    "test_record_file": "./data/test.npz",
    "train_eval_file": "./data/train_eval.json",
    "train_record_file": "./data/train.npz",
    "use_squad_v2": true,
    "word_emb_file": "./data/word_emb.json"
}
[04.02.21 00:27:43] Args: {
    "ans_limit": 30,
    "batch_size": 64,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "hidden_size": 100,
    "load_

In [4]:
import json

# word to id dictionary
word2idx_dict = json.load(open(word2idx_path))

In [5]:
# Get data loader
log.info('Building dataset...')
record_file = vars(args)[f'{args.split}_record_file']
dataset = SQuAD(record_file, args.use_squad_v2)
data_loader = data.DataLoader(dataset,
                              batch_size=args.batch_size,
                              shuffle=False,
                              num_workers=args.num_workers,
                              collate_fn=collate_fn)

[04.01.21 15:38:41] Building dataset...


In [6]:


# Set up logging
args.save_dir = util.get_save_dir(args.save_dir, args.name, training=False)
log = util.get_logger(args.save_dir, args.name)
log.info(f'Args: {dumps(vars(args), indent=4, sort_keys=True)}')
device, gpu_ids = util.get_available_devices()
args.batch_size *= max(1, len(gpu_ids))

# Get embeddings
log.info('Loading embeddings...')
word_vectors = util.torch_from_json(args.word_emb_file)

[04.01.21 15:38:44] Args: {
    "ans_limit": 30,
    "batch_size": 64,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "hidden_size": 100,
    "load_path": "save/train/baseline-01/step_1250813.pth.tar",
    "max_ans_len": 15,
    "name": "eval",
    "num_visuals": 10,
    "num_workers": 4,
    "para_limit": 400,
    "ques_limit": 50,
    "save_dir": "./save/test/eval-02/test/eval-01",
    "split": "dev",
    "sub_file": "submission.csv",
    "test_eval_file": "./data/test_eval.json",
    "test_record_file": "./data/test.npz",
    "train_eval_file": "./data/train_eval.json",
    "train_record_file": "./data/train.npz",
    "use_squad_v2": true,
    "word_emb_file": "./data/word_emb.json"
}
[04.01.21 15:38:44] Args: {
    "ans_limit": 30,
    "batch_size": 64,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "hidden_s

In [9]:
# Get model
log.info('Building model...')
model = BiDAF(word_vectors=word_vectors,
              hidden_size=args.hidden_size)
model = nn.DataParallel(model, gpu_ids)
log.info(f'Loading checkpoint from {args.load_path}...')
model = util.load_model(model, args.load_path, gpu_ids, return_step=False)
model = model.to(device)
model.eval()

[04.02.21 00:27:48] Building model...
[04.02.21 00:27:48] Building model...
[04.02.21 00:27:48] Building model...
[04.02.21 00:27:48] Loading checkpoint from save/train/baseline-01/best.pth.tar...
[04.02.21 00:27:48] Loading checkpoint from save/train/baseline-01/best.pth.tar...
[04.02.21 00:27:48] Loading checkpoint from save/train/baseline-01/best.pth.tar...


DataParallel(
  (module): BiDAF(
    (emb): Embedding(
      (embed): Embedding(88714, 300)
      (proj): Linear(in_features=300, out_features=100, bias=False)
      (hwy): HighwayEncoder(
        (transforms): ModuleList(
          (0): Linear(in_features=100, out_features=100, bias=True)
          (1): Linear(in_features=100, out_features=100, bias=True)
        )
        (gates): ModuleList(
          (0): Linear(in_features=100, out_features=100, bias=True)
          (1): Linear(in_features=100, out_features=100, bias=True)
        )
      )
    )
    (enc): RNNEncoder(
      (rnn): LSTM(100, 100, batch_first=True, bidirectional=True)
    )
    (att): BiDAFAttention()
    (mod): RNNEncoder(
      (rnn): LSTM(800, 100, num_layers=2, batch_first=True, bidirectional=True)
    )
    (out): BiDAFOutput(
      (att_linear_1): Linear(in_features=800, out_features=1, bias=True)
      (mod_linear_1): Linear(in_features=200, out_features=1, bias=True)
      (rnn): RNNEncoder(
        (rnn): 

In [10]:
# preprocessing functions
# Import spacy language model
import spacy
import numpy as np

nlp = spacy.blank("en")

def word_tokenize(sent):
    doc = nlp(sent)
    return [token.text for token in doc]

def preprocess(context, question, word2idx_dict, is_test=False):
    
    context_tokens = word_tokenize(context)
    ques_tokens = word_tokenize(question)
    
#     para_limit = args.test_para_limit if is_test else args.para_limit
#     ques_limit = args.test_ques_limit if is_test else args.ques_limit

    para_limit = args.para_limit
    ques_limit = args.ques_limit
    ans_limit = args.ans_limit
    
    example = {'context_tokens': context_tokens, 'ques_tokens': ques_tokens}
    examples = [example]
    

#     print(f"Converting {data_type} examples to indices...")
    total = 0
    total_ = 0
    meta = {}
    context_idxs = []
    context_char_idxs = []
    ques_idxs = []
    ques_char_idxs = []
    y1s = []
    y2s = []
    ids = []
    for n, example in tqdm(enumerate(examples)):
        total_ += 1


        total += 1

        def _get_word(word):
            for each in (word, word.lower(), word.capitalize(), word.upper()):
                if each in word2idx_dict:
                    return word2idx_dict[each]
            return 1


        context_idx = np.zeros([para_limit], dtype=np.int32)
#         context_char_idx = np.zeros([para_limit, char_limit], dtype=np.int32)
        ques_idx = np.zeros([ques_limit], dtype=np.int32)
#         ques_char_idx = np.zeros([ques_limit, char_limit], dtype=np.int32)

        for i, token in enumerate(example["context_tokens"]):
            context_idx[i] = _get_word(token)
#         context_idxs.append(context_idx)

        for i, token in enumerate(example["ques_tokens"]):
            ques_idx[i] = _get_word(token)
#         ques_idxs.append(ques_idx)

        
        return context_idx, ques_idx
    
def merge_1d(arrays, dtype=torch.int64, pad_value=0):
        lengths = [(a != pad_value).sum() for a in arrays]
        padded = torch.zeros(len(arrays), max(lengths), dtype=dtype)
        for i, seq in enumerate(arrays):
            end = lengths[i]
            padded[i, :end] = seq[:end]
        return padded

In [11]:
# answer custom questions

# context = '''Southern California, often abbreviated SoCal, is a geographic and cultural region that generally comprises California's southernmost 10 counties. The region is traditionally described as "eight counties", based on demographics and economic ties: Imperial, Los Angeles, Orange, Riverside, San Bernardino, San Diego, Santa Barbara, and Ventura. The more extensive 10-county definition, including Kern and San Luis Obispo counties, is also used based on historical political divisions. Southern California is a major economic center for the state of California and the United States.'''
# question = "What is Southern California often abbreviated as?"

context = "The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."
question = "In what country is Normandy located?"

# preprocess
# build_features(args, train_examples, "train", args.train_record_file, word2idx_dict, char2idx_dict)

def answer_question(context, question):
    context_idxs, ques_idxs = preprocess(context, question, word2idx_dict)

    context_idxs = np.insert(context_idxs, 0, 1)
    ques_idxs = np.insert(ques_idxs, 0, 1)

    context_idxs = np.expand_dims(context_idxs, axis=0)
    ques_idxs = np.expand_dims(ques_idxs, axis=0)


    context_idxs = torch.from_numpy(context_idxs).long()
    ques_idxs = torch.from_numpy(ques_idxs).long()

    context_idxs = merge_1d(context_idxs)
    ques_idxs = merge_1d(ques_idxs)

    # ones = torch.ones((batch_size, 1), dtype=torch.int64)
    # self.context_idxs = torch.cat((ones, self.context_idxs), dim=1)
    # self.question_idxs = torch.cat((ones, self.question_idxs), dim=1)

#     context_idxs, ques_idxs
    
    # run model

    # print(context_idxs.shape, ques_idxs.shape)

    # context_idxs, ques_idxs = data_loader.dataset[0][0], data_loader.dataset[0][2]
    log_p1, log_p2 = model(context_idxs, ques_idxs)
    p1, p2 = log_p1.exp(), log_p2.exp()
    starts, ends = util.discretize(p1, p2, args.max_ans_len, args.use_squad_v2)
    
    context_tokens = word_tokenize(context)
#     print(context_tokens[:10])

    start_idx, end_idx = starts.item(), ends.item()
    if (start_idx == 0 or end_idx == 0):
        print("no answer")
    print(context_tokens[start_idx-1:end_idx])
    
answer_question(context, question)

0it [00:00, ?it/s]

['France']


In [12]:
# retriever API

import requests

# query = "In what country is Normandy located?"
query = "What branch of theoretical computer science deals with broadly classifying computational problems by difficulty and class of relationship?"

limit = 10
r = requests.get(f"http://127.0.0.1:5000/search_passages?query={query}&n_docs={limit}")
# r = requests.get(f"http://127.0.0.1:5000/get_document_by_id/{doc_id}")
assert r.status_code == requests.codes.ok

search_results = r.json()
# print('search_results', search_results)

for res in search_results:
    print(res['page'])
    print(res['text'])

Computational complexity theory
Computational complexity theory Computational complexity theory focuses on classifying computational problems according to their inherent difficulty, and relating these classes to each other. A computational problem is a task solved by a computer. A computation problem is solvable by mechanical application of mathematical steps, such as an algorithm. A problem is regarded as inherently difficult if its solution requires significant resources, whatever the algorithm used. The theory formalizes this intuition, by introducing mathematical models of computation to study these problems and quantifying their computational complexity, i.e., the amount of resources needed to solve them, such as time and storage. Other
Computational complexity theory
widely believed that if a problem turned out to be NP-complete, then there was little chance of being able to work with the problem in a practical situation. However, it became increasingly clear that this is not alw

In [ ]:
# with open('data/dev_eval.json') as fh:
#     gold_dict = json.load(fh)
# len(gold_dict)

In [ ]:
# Evaluate
    log.info(f'Evaluating on {args.split} split...')
    nll_meter = util.AverageMeter()
    pred_dict = {}  # Predictions for TensorBoard
    sub_dict = {}   # Predictions for submission
    eval_file = vars(args)[f'{args.split}_eval_file']
    with open(eval_file, 'r') as fh:
        gold_dict = json_load(fh)
    with torch.no_grad(), \
            tqdm(total=len(dataset)) as progress_bar:
        for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in data_loader:
            # Setup for forward
            cw_idxs = cw_idxs.to(device)
            qw_idxs = qw_idxs.to(device)
            batch_size = cw_idxs.size(0)

            # Forward
            log_p1, log_p2 = model(cw_idxs, qw_idxs)
            y1, y2 = y1.to(device), y2.to(device)
            loss = F.nll_loss(log_p1, y1) + F.nll_loss(log_p2, y2)
            nll_meter.update(loss.item(), batch_size)

            # Get F1 and EM scores
            p1, p2 = log_p1.exp(), log_p2.exp()
            starts, ends = util.discretize(p1, p2, args.max_ans_len, args.use_squad_v2)

            # Log info
            progress_bar.update(batch_size)
            if args.split != 'test':
                # No labels for the test set, so NLL would be invalid
                progress_bar.set_postfix(NLL=nll_meter.avg)

            idx2pred, uuid2pred = util.convert_tokens(gold_dict,
                                                      ids.tolist(),
                                                      starts.tolist(),
                                                      ends.tolist(),
                                                      args.use_squad_v2)
            pred_dict.update(idx2pred)
            sub_dict.update(uuid2pred)

In [ ]:
for i, (cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids) in enumerate(data_loader):

    print(cw_idxs.shape)
    print(cw_idxs)
    
    if i>5: break

In [ ]:
data_loader.

In [ ]:
# Log results (except for test set, since it does not come with labels)
    if args.split != 'test':
        results = util.eval_dicts(gold_dict, pred_dict, args.use_squad_v2)
        results_list = [('NLL', nll_meter.avg),
                        ('F1', results['F1']),
                        ('EM', results['EM'])]
        if args.use_squad_v2:
            results_list.append(('AvNA', results['AvNA']))
        results = OrderedDict(results_list)

        # Log to console
        results_str = ', '.join(f'{k}: {v:05.2f}' for k, v in results.items())
        log.info(f'{args.split.title()} {results_str}')

        # Log to TensorBoard
        tbx = SummaryWriter(args.save_dir)
        util.visualize(tbx,
                       pred_dict=pred_dict,
                       eval_path=eval_file,
                       step=0,
                       split=args.split,
                       num_visuals=args.num_visuals)

In [ ]:
# pred_dict[:10]
# sub_dict
cw_idxs
